In [1]:
import numpy as np
import math
import random
from scipy.stats import poisson
import time

## Functions

In [2]:
I = 8  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

In [3]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [4]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [5]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probSize = calcExponentialLimit(beta*N)+1
calculate_p(beta, probSize, precision)


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [6]:
def calculate_v(p, beta, precision, precision_limit, n, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*n) + 1
  v = np.zeros((n+1, limit+d))
  add_v = np.zeros((n+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, n+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(n+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probSize, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [2.34555134e-59, 3.16649431e-57, 2.13738366e-55, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.89464031e-63, 4.16828205e-61, 3.00116308e-59, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.64691685e-07, 1.72525699e-07, 7.71596082e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 202)

In [7]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########



v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
calculateProbabilities(x, precision, limit, v, I, d)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.73265822e-07, 1.01134441e-06, 3.90089986e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.54239195e-17, 4.03791455e-16, 2.45398261e-15, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [3.81317139e-19, 1.26005994e-18, 4.73231789e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.04939850e-19, 1.04476157e-18, 3.44258673e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [8.24980486e-29, 4.87448309e-28, 2.84907818e-27, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 202)

In [8]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, n, no_show, beta):
  return (I * d) + tardiness - (n * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, n):
  w = np.zeros((I+1, n+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= n
  return waitingtime

In [9]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [10]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probSize, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [3 4 1 1 1 2 1 4],
Objective value: 67.18824768838877,
Prob calculation time: 0.138232 sec,
Waiting time (timer): 55.187868930714004 (0.259352 sec),
Idle time (timer): -0.14483313982805157 (0.000002 sec),
Tardiness (timer): 112.85516686017193 (0.000076 sec)



{'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.73265822e-07, 1.01134441e-06, 3.90089986e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.54239195e-17, 4.03791455e-16, 2.45398261e-15, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [3.81317139e-19, 1.26005994e-18, 4.73231789e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.04939850e-19, 1.04476157e-18, 3.44258673e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [8.24980486e-29, 4.87448309e-28, 2.84907818e-27, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 'waitingTime': 55.187868930714004,
 'idleTime': -0.14483313982805157,
 'tardiness': 112.85516686017193,
 'objVal': 67.18824768838877}

In [11]:
def createDist(arr):
    norm=sum(arr)
    dist=arr/norm
    return(dist)

##########
## TEST ##
##########

arr = np.random.choice(20, 8)
dist = createDist(arr)
arr, dist, sum(dist)


(array([14, 10,  8,  5,  6, 10,  2, 17]),
 array([0.19444444, 0.13888889, 0.11111111, 0.06944444, 0.08333333,
        0.13888889, 0.02777778, 0.23611111]),
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [12]:
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    # norm1 = sum(child1)
    # p1 = child1 / norm1
    p1 = createDist(child1)
    
    # The number of swaps should not exceed the maximum number of patients in any interval. Otherwise it might happen that the number of patients that are transfered exceeds availability, resulting in a negative amount.
    max1 = int(max(child1))
    max2 = int(max(child2))
    nSwaps = min(nSwaps, max1, max2)
    
    # Swap patients parent1 to parent 2
    i = np.random.choice(I, nSwaps, p=p1)
    for i in i:
        child1[i] += -1
        child2[i] += 1
    
    # Swap patients parent2 to parent 1
    # norm2 = sum(child2)
    # p2 = child2 / norm2
    p2 = createDist(child2)
    
    j = np.random.choice(I, nSwaps, p=p2)
    for j in j:
        child1[j] += 1
        child2[j] += -1
    return(child1, child2)

##########
## TEST ##
##########


parent1 = np.random.choice(5, I)
parent2 = np.random.choice(5, I)

print(parent1, parent2)
n = 3
children = crossover(parent1, parent2, n)
children


[1 0 2 0 4 2 3 1] [2 0 4 0 1 2 4 4]


(array([2, 0, 3, 0, 3, 1, 3, 1]), array([1, 0, 3, 0, 2, 3, 4, 4]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [13]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[2 0 3 0 3 1 3 1]
[1 0 3 0 3 1 3 2]


In [14]:
def createDist(arr):
    norm = sum(arr)
    dist = arr/norm
    return(dist)

##########
## TEST ##
##########


arr = np.random.choice(20, 8)
dist = createDist(arr)
arr, dist, sum(dist)

(array([11, 10, 17,  7,  5,  1, 19,  4]),
 array([0.14864865, 0.13513514, 0.22972973, 0.09459459, 0.06756757,
        0.01351351, 0.25675676, 0.05405405]),
 0.9999999999999999)

## Classes

Create a data object to store and generate initial data.

In [15]:
class Data:
    def __init__(self, I=40, N=8, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]

array((17, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 48})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [16]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)

##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
schedule = scheduleObject.get_schedule()
schedule

[0 0 0 0 0 0 0 0]


array([2, 3, 0, 7, 0, 2, 1, 2])

Create population of individuals

In [155]:
class Population:
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = list(map(lambda x: calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'], self.individuals))
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = 3
        mutationRate = 0.1
        try:
            self.population_dist = createDist(self.fitnessScores)
        except:
            print("Did you calculate fitness scores?")
        else:
            for i in range(self.params['popSize']):
                r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
                parent1, parent2 = self.individuals[r1], self.individuals[r2]
                child1, child2 = crossover(parent1, parent2, nSwaps)
                newIndividuals.append(mutate(child1, mutationRate))
                newIndividuals.append(mutate(child2, mutationRate))
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
                'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[[
                'precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = list(map(lambda x: calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N,
                                                                no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'], newIndividuals))
            tempList = list(newIndividuals)
            zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
            self.fitnessScore = zipped[0][:5]
            self.individuals = zipped[1][:5]
            print(self.fitnessScore, '\n', self.individuals)


## Tests

In [156]:
##########
## TEST ##
##########

dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0],
Objective value: 25.97148217756984,
Prob calculation time: 0.228312 sec,
Waiting time (timer): 0.7429643551396784 (0.000984 sec),
Idle time (timer): 128.0 (0.000003 sec),
Tardiness (timer): 2.6868626546784785e-23 (0.000053 sec)

Schedule: [0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0],
Objective value: 26.27997121831234,
Prob calculation time: 0.229710 sec,
Waiting time (timer): 1.3178722197596466 (0.016314 sec),
Idle time (timer): 128.04207021686503 (0.000003 sec),
Tardiness (timer): 0.04207021686502714 (0.000056 sec)

Schedule: [0 0 0 0 0 1 0 0 0 0 0 0 3 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0],
Objective value: 27.380829525069796,
Prob calculation time: 0.250793 sec,
Waiting time (timer): 3.5195888332745597 (0.023088 sec),
Idle time (timer): 128.04207021686503 (0.000005 sec),
Tardiness (timer): 0.04207021686502715 (0.000075 sec)

Schedule: 

In [157]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0]), array([0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])] 
 [array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3

In [158]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0],
Objective value: 25.97148217756984,
Prob calculation time: 0.262272 sec,
Waiting time (timer): 0.7429643551396784 (0.001253 sec),
Idle time (timer): 128.0 (0.000004 sec),
Tardiness (timer): 2.6868626546784785e-23 (0.000063 sec)

Schedule: [0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0],
Objective value: 26.27997121831234,
Prob calculation time: 0.252989 sec,
Waiting time (timer): 1.3178722197596466 (0.012200 sec),
Idle time (timer): 128.04207021686503 (0.000002 sec),
Tardiness (timer): 0.04207021686502714 (0.000048 sec)

Schedule: [0 0 0 0 0 1 0 0 0 0 0 0 3 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0],
Objective value: 27.380829525069796,
Prob calculation time: 0.212016 sec,
Waiting time (timer): 3.5195888332745597 (0.018906 sec),
Idle time (timer): 128.04207021686503 (0.000005 sec),
Tardiness (timer): 0.04207021686502715 (0.000054 sec)

Schedule: 

(array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]))

In [159]:
# Initial population
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals
    


Schedule: [0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0],
Objective value: 26.515960840718883,
Prob calculation time: 0.204258 sec,
Waiting time (timer): 0.612042800308638 (0.001000 sec),
Idle time (timer): 129.21987888112912 (0.000003 sec),
Tardiness (timer): 1.2198788811291295 (0.000052 sec)

Schedule: [1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0
 0 0 1],
Objective value: 27.946346280164654,
Prob calculation time: 0.198501 sec,
Waiting time (timer): 0.6120625626631332 (0.000980 sec),
Idle time (timer): 132.08062999766616 (0.000002 sec),
Tardiness (timer): 4.080629997666177 (0.000042 sec)

Schedule: [0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0],
Objective value: 26.661640891627634,
Prob calculation time: 0.223353 sec,
Waiting time (timer): 2.1232817832552664 (0.000873 sec),
Idle time (timer): 128.0 (0.000003 sec),
Tardiness (timer): 0.0 (0.000046 sec)

Schedule: [2 0 0 0 0 0 1 0 0 0 